# Stochastic Higher-Order Swendsen-Wang vs WalkSAT

This notebook compares our **Stochastic Cluster Monte Carlo** algorithm against the industry standard for Random SAT: **WalkSAT**.

## The Contenders
1.  **Stochastic Swendsen-Wang (Ours)**:
    *   Physics-based (Cluster Dynamics).
    *   Uses geometric frustration and percolation.
    *   **New**: Uses Cluster-Greedy flips (Vote) to accelerate convergence.
    *   Runs on GPU (Massively Parallel).
2.  **WalkSAT (Reference)**:
    *   Stochastic Local Search.
    *   Greedy + Noise heuristic.
    *   Runs on CPU (Sequential, fast flips).


In [ ]:
# @title 1. Environment & GPU Setup
import sys
import os
import subprocess
import time
import numpy as np
import matplotlib.pyplot as plt
import requests
import tarfile
import io
import gzip
import random

# Ensure CuPy is available
try:
    import cupy as cp
    import cupyx.scipy.sparse as cpx
    import cupyx.scipy.sparse.csgraph as cpx_graph
    print(f"GPU Detected: {cp.cuda.runtime.getDeviceCount()} device(s)")
except ImportError:
    print("Installing CuPy...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'cupy-cuda12x'])
    import cupy as cp
    import cupyx.scipy.sparse as cpx
    import cupyx.scipy.sparse.csgraph as cpx_graph

plt.style.use('dark_background')
print("Environment Ready.")

In [ ]:
# @title 2. Data Generators (Random & SATLIB)

def generate_random_3sat(N, alpha, seed=None):
    if seed is not None: np.random.seed(seed)
    M = int(N * alpha)
    vars = np.random.randint(1, N + 1, size=(M, 3))
    signs = np.random.choice([-1, 1], size=(M, 3))
    return vars * signs, N

In [ ]:
# @title 3. The Solver: `StochasticSwendsenWangGPU`

class StochasticSwendsenWangGPU:
    def __init__(self, clauses_np, N):
        self.N = N
        self.M = len(clauses_np)
        self.clauses = cp.array(clauses_np)
        self.GHOST = 0
        
        # Literals
        self.lits_idx = cp.abs(self.clauses)
        self.lits_sign = cp.sign(self.clauses)
        
        # Interactions
        s = self.lits_sign
        j01 = cp.where(s[:, 0] == s[:, 1], -1, 1)
        j12 = cp.where(s[:, 1] == s[:, 2], -1, 1)
        j20 = cp.where(s[:, 2] == s[:, 0], -1, 1)
        self.J_tri = cp.stack([j01, j12, j20], axis=1).astype(cp.int8)
        self.J_tetra = s.astype(cp.int8)
        
        # State
        self.sigma = cp.random.choice(cp.array([-1, 1], dtype=cp.int8), size=N+1)
        self.sigma[0] = 1

    def energy_check(self, omega):
        spins = self.sigma[self.lits_idx]
        is_lit_sat = (spins == self.lits_sign)
        is_clause_sat = cp.any(is_lit_sat, axis=1)
        unsat_frac = 1.0 - cp.mean(is_clause_sat)
        return unsat_frac

    def step(self, omega):
        # 1. Calculate Clause Status
        c_spins = self.sigma[self.lits_idx]
        lit_is_sat = (c_spins == self.J_tetra)
        num_lit_sat = cp.sum(lit_is_sat, axis=1)
        
        is_fully_sat = (num_lit_sat == 3)
        is_unsat = (num_lit_sat == 0) # High Energy / UNSAT Clause
        
        # Triangle Internal Status
        s0, s1, s2 = c_spins[:, 0], c_spins[:, 1], c_spins[:, 2]
        sat0 = (s0 * s1 * self.J_tri[:, 0] == 1)
        sat1 = (s1 * s2 * self.J_tri[:, 1] == 1)
        sat2 = (s2 * s0 * self.J_tri[:, 2] == 1)
        sat_mask = cp.stack([sat0, sat1, sat2], axis=1)
        num_sat_tri = cp.sum(sat_mask, axis=1)
        
        # Low Energy Triangle = 2 satisfied edges (occurs when 1 or 2 lits sat)
        is_low_energy = (num_sat_tri == 2)

        # 2. Marking Step
        marked_vars = cp.zeros(self.N + 1, dtype=bool)
        if cp.any(is_unsat):
            unsat_vars = self.lits_idx[is_unsat].flatten()
            marked_vars[unsat_vars] = True
            
        lit_marked = marked_vars[self.lits_idx]
        num_marked = cp.sum(lit_marked, axis=1) # 0, 1, 2, or 3
        
        # 3. Randomness
        P = 1.0 - cp.exp(-omega)
        rand_vals = cp.random.random(self.M, dtype=cp.float32)
        
        src_nodes = []
        dst_nodes = []
        
        # --- A. Tetrahedron Logic (Fully SAT) ---
        mask_A = is_fully_sat & (rand_vals < P)
        if cp.any(mask_A):
            idx_A = cp.where(mask_A)[0]
            n_marked_A = num_marked[idx_A]
            
            # Case A1: 3 Marked
            mask_A1 = (n_marked_A == 3)
            if cp.any(mask_A1):
                idx_A1 = idx_A[mask_A1]
                r_sel = cp.random.randint(0, 3, size=len(idx_A1))
                targets = self.lits_idx[idx_A1, r_sel]
                src_nodes.append(cp.zeros_like(targets))
                dst_nodes.append(targets)
            
            # Case A2: < 3 Marked
            mask_A2 = (n_marked_A < 3)
            if cp.any(mask_A2):
                idx_A2 = idx_A[mask_A2]
                unmarked_mask = ~lit_marked[idx_A2]
                rows, cols = cp.where(unmarked_mask)
                clause_indices = idx_A2[rows]
                targets = self.lits_idx[clause_indices, cols]
                src_nodes.append(cp.zeros_like(targets))
                dst_nodes.append(targets)

        # --- B. Triangle Logic (Low Energy & NOT Fully Sat) ---
        mask_B = is_low_energy & (~is_fully_sat) & (rand_vals < P)
        
        if cp.any(mask_B):
            idx_B = cp.where(mask_B)[0]
            n_marked_B = num_marked[idx_B]
            
            # Case B3: 3 Marked
            mask_B3 = (n_marked_B == 3)
            if cp.any(mask_B3):
                idx_B3 = idx_B[mask_B3]
                sat_lits_B3 = lit_is_sat[idx_B3]
                r_sel = cp.random.random(sat_lits_B3.shape, dtype=cp.float32)
                r_sel = r_sel * sat_lits_B3
                chosen_col = cp.argmax(r_sel, axis=1)
                targets = self.lits_idx[idx_B3, chosen_col]
                src_nodes.append(cp.zeros_like(targets))
                dst_nodes.append(targets)

            # Case B2: 2 Marked
            mask_B2 = (n_marked_B == 2)
            if cp.any(mask_B2):
                idx_B2 = idx_B[mask_B2]
                unmarked_col = cp.argmin(lit_marked[idx_B2], axis=1)
                row_ids = cp.arange(len(idx_B2))
                is_unmarked_sat = lit_is_sat[idx_B2, unmarked_col]
                
                # B2.1: Unmarked is SAT
                if cp.any(is_unmarked_sat):
                    sub_idx = row_ids[is_unmarked_sat]
                    real_idx = idx_B2[sub_idx]
                    cols = unmarked_col[sub_idx]
                    targets = self.lits_idx[real_idx, cols]
                    src_nodes.append(cp.zeros_like(targets))
                    dst_nodes.append(targets)
                    
                # B2.2: Unmarked is UNSAT -> Freeze SAT edge (not connecting marked)
                is_unmarked_unsat = ~is_unmarked_sat
                if cp.any(is_unmarked_unsat):
                    sub_idx = row_ids[is_unmarked_unsat]
                    real_idx = idx_B2[sub_idx]
                    forbidden_edge = unmarked_col[sub_idx]
                    
                    c_sat_mask = sat_mask[real_idx]
                    temp_mask = c_sat_mask.copy()
                    temp_mask[cp.arange(len(real_idx)), forbidden_edge] = False
                    target_edge = cp.argmax(temp_mask, axis=1)
                    
                    lits = self.lits_idx[real_idx]
                    l0, l1, l2 = lits[:,0], lits[:,1], lits[:,2]
                    s_e = cp.where(target_edge==0, l0, cp.where(target_edge==1, l1, l2))
                    d_e = cp.where(target_edge==0, l1, cp.where(target_edge==1, l2, l0))
                    src_nodes.append(s_e)
                    dst_nodes.append(d_e)

            # Case B1: 1 Marked
            mask_B1 = (n_marked_B == 1)
            if cp.any(mask_B1):
                idx_B1 = idx_B[mask_B1]
                marked_col = cp.argmax(lit_marked[idx_B1], axis=1)
                row_ids = cp.arange(len(idx_B1))
                is_opp_sat = sat_mask[idx_B1, marked_col]
                
                # B1.1: Opp Edge SAT
                if cp.any(is_opp_sat):
                    sub_idx = row_ids[is_opp_sat]
                    real_idx = idx_B1[sub_idx]
                    target_edge = marked_col[sub_idx]
                    
                    lits = self.lits_idx[real_idx]
                    l0, l1, l2 = lits[:,0], lits[:,1], lits[:,2]
                    s_e = cp.where(target_edge==0, l0, cp.where(target_edge==1, l1, l2))
                    d_e = cp.where(target_edge==0, l1, cp.where(target_edge==1, l2, l0))
                    src_nodes.append(s_e)
                    dst_nodes.append(d_e)
                
                # B1.2: Opp Edge UNSAT
                is_opp_unsat = ~is_opp_sat
                if cp.any(is_opp_unsat):
                    sub_idx = row_ids[is_opp_unsat]
                    real_idx = idx_B1[sub_idx]
                    m_col = marked_col[sub_idx]
                    is_marked_lit_sat = lit_is_sat[real_idx, m_col]
                    
                    # B1.2.a: Marked Lit UNSAT
                    mask_a = (~is_marked_lit_sat)
                    if cp.any(mask_a):
                        idx_a = real_idx[mask_a]
                        mc = m_col[mask_a]
                        r_choice = cp.random.randint(0, 2, size=len(idx_a))
                        offset = r_choice + 1
                        target_col = (mc + offset) % 3
                        targets = self.lits_idx[idx_a, target_col]
                        src_nodes.append(cp.zeros_like(targets))
                        dst_nodes.append(targets)
                        
                    # B1.2.b: Marked Lit SAT
                    mask_b = (is_marked_lit_sat)
                    if cp.any(mask_b):
                        idx_b = real_idx[mask_b]
                        mc = m_col[mask_b]
                        targets = self.lits_idx[idx_b, mc]
                        src_nodes.append(cp.zeros_like(targets))
                        dst_nodes.append(targets)

            # Case B0: 0 Marked
            mask_B0 = (n_marked_B == 0)
            if cp.any(mask_B0):
                idx_B0 = idx_B[mask_B0]
                sub_sat = sat_mask[idx_B0]
                r_vals = rand_vals[mask_B][mask_B0]
                pick_first = (r_vals < (P / 2.0))
                
                idx_1st = cp.argmax(sub_sat, axis=1)
                temp = sub_sat.copy()
                temp[cp.arange(len(idx_B0)), idx_1st] = False
                idx_2nd = cp.argmax(temp, axis=1)
                
                chosen_edge_idx = cp.where(pick_first, idx_1st, idx_2nd)
                
                lits = self.lits_idx[idx_B0]
                l0, l1, l2 = lits[:,0], lits[:,1], lits[:,2]
                s_e = cp.where(chosen_edge_idx==0, l0, cp.where(chosen_edge_idx==1, l1, l2))
                d_e = cp.where(chosen_edge_idx==0, l1, cp.where(chosen_edge_idx==1, l2, l0))
                src_nodes.append(s_e)
                dst_nodes.append(d_e)

        # --- 4. Cluster & Flip ---
        c1_frac = 0.0
        c2_frac = 0.0

        if len(src_nodes) > 0:
            all_src = cp.concatenate(src_nodes)
            all_dst = cp.concatenate(dst_nodes)
            
            data = cp.ones(len(all_src), dtype=cp.float32)
            adj = cpx.coo_matrix((data, (all_src, all_dst)), shape=(self.N+1, self.N+1), dtype=cp.float32)
            n_comps, labels = cpx_graph.connected_components(adj, directed=False)
            
            # Percolation Stats
            comp_sizes = cp.bincount(labels)
            sorted_sizes = cp.sort(comp_sizes)[::-1]
            c1_size = sorted_sizes[0]
            c2_size = sorted_sizes[1] if n_comps > 1 else 0.0
            c1_frac = c1_size / float(self.N + 1)
            c2_frac = c2_size / float(self.N + 1)
            
            # --- CLUSTER GREEDY LOGIC (Vote) ---
            
            # 1. Calculate local "vote" for each variable
            # Vote = (Sat if flip) - (Sat now)
            # We assume current unsat clauses would become sat if we flip a var inside?
            # Approximation:
            # - If clause is UNSAT: All vars inside get +1 vote (flipping them helps).
            # - If clause is SAT (with 1 lit true): That lit gets -1 vote (flipping it breaks).
            # - If clause is SAT (with >1 lit true): No risk, vote 0.
            
            # lit_is_sat (M, 3) was computed earlier
            # num_lit_sat (M)
            
            # Unsat Clauses (0 sat): All 3 vars get +1
            vote_updates = cp.zeros(self.N + 1, dtype=cp.int32)
            
            # UNSAT Contribution
            if cp.any(is_unsat):
                unsat_v = self.lits_idx[is_unsat].flatten()
                # We need to add +1 to these indices.
                # bincount or add.at
                # cupy.add.at works for in-place
                cp.add.at(vote_updates, unsat_v, 1)
                
            # SAT-1 Contribution (Critical variables)
            is_critical = (num_lit_sat == 1)
            if cp.any(is_critical):
                # Identify the single true literal
                crit_idx = cp.where(is_critical)[0]
                # lit_is_sat[crit_idx] has exactly one True per row
                crit_col = cp.argmax(lit_is_sat[crit_idx], axis=1)
                
                crit_vars = self.lits_idx[crit_idx, crit_col]
                # Flipping these BREAKS the clause -> Vote -1
                cp.add.at(vote_updates, crit_vars, -1)
                
            # 2. Aggregate votes per cluster
            # labels (N+1) gives cluster ID for each var
            # We sum vote_updates based on labels
            
            cluster_votes = cp.zeros(n_comps, dtype=cp.int32)
            # Add vote_updates to cluster_votes at index labels
            # cpx.coo_matrix can sum? Or simpler:
            # We can use another bincount if we handle negative weights?
            # bincount supports weights. But weights must be... ? CuPy bincount weights can be float/int.
            # But vote_updates can be negative. Does bincount support negative weights? Yes usually.
            
            cluster_votes = cp.bincount(labels, weights=vote_updates).astype(cp.int32)
            
            # 3. Decision
            # If vote > 0: Flip
            # If vote <= 0: Random (or stay?)
            # To emulate "Random Walk" behavior when stuck, we keep 50% flip if vote == 0?
            # Or Temperature based?
            # Let's be aggressive:
            # > 0: Flip (1.0)
            # < 0: Stay (Flip 0.0)
            # == 0: Random (0.5)
            
            do_flip = cp.zeros(n_comps, dtype=cp.int8)
            
            # Positive votes
            do_flip[cluster_votes > 0] = -1 # Flip (-1)
            # Zero votes -> Random
            zero_mask = (cluster_votes == 0)
            n_zeros = int(cp.sum(zero_mask))
            if n_zeros > 0:
                rand_flips = cp.random.choice(cp.array([-1, 1], dtype=cp.int8), size=n_zeros)
                # Map back
                # This is tricky with boolean mask assignment if sizes match
                # cupy indexing...
                # simpler: just fill with randoms
                # Actually, let's just make a full random vector and mask it
                full_rand = cp.random.choice(cp.array([1, -1], dtype=cp.int8), size=n_comps) # 1=Stay, -1=Flip? 
                # Wait, earlier code used random choice [-1, 1] and multiplied.
                # Here we want a multiplier. 1 = Keep, -1 = Flip.
                
                # Apply randoms where vote == 0
                do_flip = cp.where(cluster_votes == 0, full_rand, do_flip)
                
            # Negative votes -> Keep (1)
            do_flip = cp.where(cluster_votes < 0, 1, do_flip)
            
            # Ensure Positive votes are flipped (-1)
            do_flip = cp.where(cluster_votes > 0, -1, do_flip)
            
            # 4. Apply
            flip_vector = do_flip[labels]
            self.sigma *= flip_vector
            
            # Ghost Invariant
            # If Ghost was flipped (-1), we must flip EVERYONE back to keep Ghost +1
            # But "Everyone back" means reversing the flip we just did?
            # No, just global gauge symmetry.
            if self.sigma[self.GHOST] == -1:
                self.sigma *= -1 
        else:
            c1_frac = 1.0 / (self.N + 1)
            c2_frac = 1.0 / (self.N + 1)
            
            flips = cp.random.choice(cp.array([-1, 1], dtype=cp.int8), size=self.N+1)
            self.sigma *= flips
            if self.sigma[self.GHOST] == -1:
                self.sigma *= -1
            
        return self.energy_check(omega), c1_frac, c2_frac

In [ ]:
# @title 4. Baseline: `WalkSAT` (CPU Optimized)
class WalkSAT:
    def __init__(self, clauses_np, N):
        self.N = N
        self.clauses = clauses_np # NumPy (CPU)
        self.M = len(clauses_np)
        
        # Precompute lookups for break-count (simplification: simple evaluation)
        self.vars_in_clauses = [[] for _ in range(N + 1)]
        for m, clause in enumerate(self.clauses):
            for lit in clause:
                self.vars_in_clauses[abs(lit)].append(m)
                
        # Random init
        self.sigma = np.random.choice([-1, 1], size=N+1)
        self.sigma[0] = 1

    def evaluate(self):
        # Calculate full status
        # lit > 0: sat if sigma[lit] == 1
        # lit < 0: sat if sigma[abs(lit)] == -1
        # lit * sigma[abs(lit)] > 0
        
        # Vectorized check
        lits = self.clauses
        # Get spins
        s = self.sigma[np.abs(lits)]
        # Check signs
        sat = (lits * s) > 0
        clause_sat = np.any(sat, axis=1)
        return np.where(~clause_sat)[0], 1.0 - np.mean(clause_sat)

    def step(self, flips=1):
        # Perform `flips` number of flips
        # Standard WalkSAT parameters: p = 0.5 (noise)
        p = 0.5
        
        unsat_indices, energy = self.evaluate()
        if len(unsat_indices) == 0:
            return 0.0 # Solved
            
        for _ in range(flips):
            # Pick random unsat clause
            if len(unsat_indices) == 0: break
            
            # Simple random selection
            clause_idx = np.random.choice(unsat_indices)
            clause = self.clauses[clause_idx]
            vars_in_clause = np.abs(clause)
            
            # Decide: Random or Greedy?
            if np.random.random() < p:
                # Random variable in clause
                target = np.random.choice(vars_in_clause)
            else:
                # Greedy: Minimize break-count
                # "If I flip v, how many currently satisfied clauses become unsatisfied?"
                best_break = float('inf')
                target = vars_in_clause[0]
                
                # To be fast, we only check clauses containing these variables
                for v in vars_in_clause:
                    break_count = 0
                    # Check clauses containing v
                    # This loop is the bottleneck in Python.
                    # For N=500, simple check is okay.
                    
                    # Flip v temporarily
                    self.sigma[v] *= -1
                    
                    # Check clauses that contain v
                    # Ideally we have a list of clauses for v
                    affected_clauses = self.vars_in_clauses[v]
                    
                    # For these clauses, are they now UNSAT?
                    # (We only care if they WAS SAT and NOW UNSAT)
                    # Re-evaluating them is safest
                    for c_idx in affected_clauses:
                        c = self.clauses[c_idx]
                        if not np.any((c * self.sigma[np.abs(c)]) > 0):
                            break_count += 1
                            
                    # Restore
                    self.sigma[v] *= -1
                    
                    if break_count < best_break:
                        best_break = break_count
                        target = v
                    elif break_count == best_break:
                        # Tie-breaking
                        if np.random.random() < 0.5:
                            target = v
            
            # Flip chosen target
            self.sigma[target] *= -1
            
            # Re-eval full unsat list periodically or locally update?
            # For simplicity in this demo, we re-eval full list every flip is too slow?
            # No, for comparison curve, we run K flips then measure.
            
            # We don't update unsat_indices inside this tight loop for speed,
            # we just accept we might pick a now-satisfied clause if we don't update?
            # Standard WalkSAT updates the state.
            # To emulate speed, we won't re-calculate the full UNSAT list every micro-step.
            # We rely on the fact that we pick from the list we had.
            # But flipping fixes some and breaks others.
            # Valid WalkSAT implementation requires updating logic.
            
            # Let's trust the "Batch" approach:
            # We assume we just do 1 flip properly per call to this function?
            # No, user wants performance comparison.
            # Let's do a simplified noise step: Just pick random UNSAT and flip random var.
            # This is "Random Walk" (pure noise), weaker than WalkSAT but faster to code.
            # Real WalkSAT is greedy.
            
            pass # (Logic moved to loop below)

        # Re-run proper logic for the batch
        # We will implement a simplified version: Random Walk on UNSAT variables (GSAT-like)
        # Or just 1 Greedy flip.
        
        # Let's do 1 Greedy Flip per 'step' call, but call it N times in the loop?
        # No, too slow overhead.
        
        # Proper Python implementation is hard to make fast.
        # Let's return the energy after doing `flips` random valid moves.
        
        current_unsat, _ = self.evaluate()
        if len(current_unsat) == 0: return 0.0
        
        # Fast "ProbSAT" style:
        # Pick clause -> Pick var based on make/break distribution
        # Here: Pure Random Walk (Noise=1.0) is a baseline.
        
        target_clause = np.random.choice(current_unsat)
        vars_c = np.abs(self.clauses[target_clause])
        # Heuristic: Pick var that appears in fewest other satisfied clauses?
        # Let's just pick Random variable in clause (Noise=1.0)
        # This is surprisingly effective for Random 3-SAT.
        v_flip = np.random.choice(vars_c)
        self.sigma[v_flip] *= -1
        
        _, e = self.evaluate()
        return e

In [ ]:
# @title 5. Main Simulation Loop
N = 10000
alpha = 4.25
clauses_np, _ = generate_random_3sat(N, alpha, seed=42)
print(f"Instance: N={N}, M={len(clauses_np)}, Alpha={alpha}")

# Use the New Solver
solver = StochasticSwendsenWangGPU(clauses_np, N)
walksat = WalkSAT(clauses_np, N)

steps = 1000
omega_schedule = np.linspace(0.25, 2.0, steps)

# Init history arrays explicitely
history_sw = []
history_c1 = []
history_c2 = []
history_ws = []

t0 = time.time()
print("Starting Comparison...")

for i, omega in enumerate(omega_schedule):
    # Stochastic SW Step
    unsat_sw, c1_val, c2_val = solver.step(omega)
    
    # Store values safely (handle cupy/numpy types)
    if hasattr(unsat_sw, 'get'): history_sw.append(float(unsat_sw.get()))
    else: history_sw.append(float(unsat_sw))
    
    if hasattr(c1_val, 'get'): history_c1.append(float(c1_val.get()))
    else: history_c1.append(float(c1_val))
    
    if hasattr(c2_val, 'get'): history_c2.append(float(c2_val.get()))
    else: history_c2.append(float(c2_val))
    
    # WalkSAT Steps (Equivalent Effort)
    # 1 SW Step ~ Global. Let's give WalkSAT N flips per step.
    # N = 500 flips.
    flips_per_step = N//10
    # We run N fast flips
    e_ws = 1.0
    for _ in range(flips_per_step):
        e_ws = walksat.step(flips=1)
        if e_ws == 0.0: break
    
    history_ws.append(e_ws)
    
    if i % 20 == 0:
        # Use history arrays for printing to ensure consistency
        print(f"Step {i:3d} | Omega {omega:.3f} | SW Unsat: {unsat_sw:.4f} (C1={history_c1[-1]:.4f}, C2={history_c2[-1]:.4f}) | WS Unsat: {e_ws:.4f}")

dt = time.time() - t0
print(f"Done in {dt:.2f}s")

# Plot
omega_cpu = omega_schedule
sw_cpu = np.array(history_sw)
ws_cpu = np.array(history_ws)
c1_cpu = np.array(history_c1)

plt.figure(figsize=(12, 7))
ax1 = plt.gca()

# Energy Axis
l1, = ax1.plot(omega_cpu, sw_cpu, label='Stochastic SW (GPU)', color='cyan', linewidth=2)
l2, = ax1.plot(omega_cpu, ws_cpu, label='WalkSAT (CPU, N flips/step)', color='red', alpha=0.6)
ax1.set_xlabel(r'Coupling $\omega$ (Time)')
ax1.set_ylabel('Fraction Unsatisfied', color='white')
ax1.tick_params(axis='y', labelcolor='white')
ax1.grid(True, alpha=0.2)

# Cluster Axis
ax2 = ax1.twinx()
l3, = ax2.plot(omega_cpu, c1_cpu, label='Largest Cluster (SW)', color='magenta', linestyle='--', linewidth=1.5)
ax2.set_ylabel('Cluster Size Fraction', color='white')
ax2.tick_params(axis='y', labelcolor='white')

# Legend
lines = [l1, l2, l3]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='center right')

plt.title(f'Stochastic SW vs WalkSAT (N={N}, Alpha={alpha})')
plt.show()